iterate over file directories: https://www.geeksforgeeks.org/how-to-iterate-over-files-in-directory-using-python/

import local module: https://stackoverflow.com/questions/53049195/importing-custom-module-into-jupyter-notebook/53052990

\_\_init\_\_ files: https://stackoverflow.com/questions/1944569/how-do-i-write-good-correct-package-init-py-files

In [1]:
import os
import glob
import sys
import numpy as np
import pandas as pd
sys.path.append('../src/')
from util import hdf5_getters as hget
import h5py
import tables

In [2]:
rootdir = '/Users/jasonmiller/Documents/docs/Northeastern/2021_2SU/CS6220/final_project/datasets/MillionSongSubset'
reldir = '../datasets/MillionSongSubset'

In [106]:
# returns list of all files with extension .h5 in a directory
# iterates through subdirectories as well
def load_all_h5_files(rootdir, ext='.h5'):
    tab = []
    for subdir, dirs, files in os.walk(rootdir):
        files = glob.glob(os.path.join(subdir,'*'+ext))
        for f in files:
            tab.append(f)
    return tab

def get_all_titles(rootdir, ext='.h5'):
    titles = []
    for subdir, dirs, files in os.walk(rootdir):
        files = glob.glob(os.path.join(subdir,'*'+ext))
        for f in files:
            h5 = hget.open_h5_file_read(f)
            titles.append(hget.get_title(h5))
            h5.close()
    return titles

# checked for multiple songs - none in the subset
def get_files_with_multiple_songs(rootdir, ext='.h5'):
    h5s = []
    for subdir, dirs, files in os.walk(rootdir):
        files = glob.glob(os.path.join(subdir,'*'+ext))
        for f in files:
            h5 = hget.open_h5_file_read(f)
            if ( hget.get_num_songs(h5) > 1 ):
                h5s.append(f)
            h5.close()
    return h5s

In [5]:
files = load_all_h5_files(reldir)

In [105]:
# want to make sure each h5 has 3 groups: 'analysis', 'metadata', 'musicbrainz'
def assert_rootgroup_contains_same_number_groups(files):
    violations = []
    for file in files:
        h5 = hget.open_h5_file_read(file)
        if ( len(h5.root._f_list_nodes()) > 3 ):
            violations.append(h5)
        h5.close()
    return violations

violations = assert_rootgroup_contains_same_number_groups(files)
len(violations)

0

In [144]:
h5.root.analysis.bars_confidence

/analysis/bars_confidence (EArray(134,), shuffle, zlib(1)) 'array of confidence of bars'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (1024,)

Each h5 file has a root with groups: ('analysis', 'metadata', 'musicbrainz')

Can get a dictionary of all children in each group using `._v_children`

In [119]:
h5.root.analysis.bars_confidence._v_title

'array of confidence of bars'

In [137]:
h5.root.analysis._v_children.keys()

dict_keys(['bars_confidence', 'bars_start', 'beats_confidence', 'beats_start', 'sections_confidence', 'sections_start', 'segments_confidence', 'segments_loudness_max', 'segments_loudness_max_time', 'segments_loudness_start', 'segments_pitches', 'segments_start', 'segments_timbre', 'songs', 'tatums_confidence', 'tatums_start'])

In [139]:
for key in h5.root.analysis._v_children.keys():
    print(key)

bars_confidence
bars_start
beats_confidence
beats_start
sections_confidence
sections_start
segments_confidence
segments_loudness_max
segments_loudness_max_time
segments_loudness_start
segments_pitches
segments_start
segments_timbre
songs
tatums_confidence
tatums_start


In [132]:
h5.root.analysis._v_children.__getitem__('bars_confidence')

/analysis/bars_confidence (EArray(134,), shuffle, zlib(1)) 'array of confidence of bars'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (1024,)

In [120]:
dir(h5.root.analysis._v_children)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_container',
 '_get_value_from_container',
 'clear',
 'containerref',
 'copy',
 'fromkeys',
 'get',
 'items',
 'iteritems',
 'itervalues',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [146]:
h5 = hget.open_h5_file_read(files[61])
h5.root._v_groups
h5.root._f_iter_nodes()
for node in h5.root._f_iter_nodes():
    for array in node._f_iter_nodes():
        print(array)

/analysis/bars_confidence (EArray(123,), shuffle, zlib(1)) 'array of confidence of bars'
/analysis/bars_start (EArray(123,), shuffle, zlib(1)) 'array of start times of bars'
/analysis/beats_confidence (EArray(497,), shuffle, zlib(1)) 'array of confidence of sections'
/analysis/beats_start (EArray(497,), shuffle, zlib(1)) 'array of start times of beats'
/analysis/sections_confidence (EArray(10,), shuffle, zlib(1)) 'array of confidence of sections'
/analysis/sections_start (EArray(10,), shuffle, zlib(1)) 'array of start times of sections'
/analysis/segments_confidence (EArray(885,), shuffle, zlib(1)) 'array of confidence of segments'
/analysis/segments_loudness_max (EArray(885,), shuffle, zlib(1)) 'array of max loudness of segments'
/analysis/segments_loudness_max_time (EArray(885,), shuffle, zlib(1)) 'array of max loudness time of segments'
/analysis/segments_loudness_start (EArray(885,), shuffle, zlib(1)) 'array of loudness of segments at start time'
/analysis/segments_pitches (EArray(

In [4]:
def get_in_files(files, getter_func):
    data = []
    for f in files:
        h5 = hget.open_h5_file_read(f)
        data.append(getter_func(h5))
        h5.close()
    return data

In [26]:
h5 = hget.open_h5_file_read(files[690])
h = h5.root.musicbrainz
#print(h)
#dir(h)
#type(h)
for node in h._f_walknodes():
    print(node)
    print(type(node))

h5.close()

/musicbrainz/artist_mbtags (EArray(0,), shuffle, zlib(1)) 'array of tags from MusicBrainz for an artist'
<class 'tables.earray.EArray'>
/musicbrainz/artist_mbtags_count (EArray(0,), shuffle, zlib(1)) 'array of tag counts from MusicBrainz for an artist'
<class 'tables.earray.EArray'>
/musicbrainz/songs (Table(1,), shuffle, zlib(1)) 'table of data coming from MusicBrainz'
<class 'tables.table.Table'>


In [99]:
h5 = hget.open_h5_file_read(files[460])
print(dir(h5.root))

['_AttributeSet', '__class__', '__contains__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__members__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_c_classid', '_c_lazy_children_attrs', '_f_close', '_f_copy', '_f_copy_children', '_f_delattr', '_f_flush', '_f_get_child', '_f_getattr', '_f_isvisible', '_f_iter_nodes', '_f_list_nodes', '_f_move', '_f_remove', '_f_rename', '_f_setattr', '_f_walk_groups', '_f_walknodes', '_g_add_children_names', '_g_check_group', '_g_check_has_child', '_g_check_name', '_g_check_not_contains', '_g_check_open', '_g_close', '_g_close_descendents', '_g_close_group', '_g_copy', '_g_copy_as_child', '_g_copy_children', '_g_create', '_g_del_location', '_g

In [80]:
titles = get_in_files(files, getter_func=hget.get_title)

In [147]:
getters = dir(hget)[8:-2]
getters

['get_analysis_sample_rate',
 'get_artist_7digitalid',
 'get_artist_familiarity',
 'get_artist_hotttnesss',
 'get_artist_id',
 'get_artist_latitude',
 'get_artist_location',
 'get_artist_longitude',
 'get_artist_mbid',
 'get_artist_mbtags',
 'get_artist_mbtags_count',
 'get_artist_name',
 'get_artist_playmeid',
 'get_artist_terms',
 'get_artist_terms_freq',
 'get_artist_terms_weight',
 'get_audio_md5',
 'get_bars_confidence',
 'get_bars_start',
 'get_beats_confidence',
 'get_beats_start',
 'get_danceability',
 'get_duration',
 'get_end_of_fade_in',
 'get_energy',
 'get_key',
 'get_key_confidence',
 'get_loudness',
 'get_mode',
 'get_mode_confidence',
 'get_num_songs',
 'get_release',
 'get_release_7digitalid',
 'get_sections_confidence',
 'get_sections_start',
 'get_segments_confidence',
 'get_segments_loudness_max',
 'get_segments_loudness_max_time',
 'get_segments_loudness_start',
 'get_segments_pitches',
 'get_segments_start',
 'get_segments_timbre',
 'get_similar_artists',
 'get_so

In [49]:
getters[0]

'get_analysis_sample_rate'

In [50]:
get_in_files(files, hget.getters[0])

AttributeError: module 'util.hdf5_getters' has no attribute 'getters'